In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
!ls ../input/keras-pretrained-models/

In [ ]:
cache_dir = os.path.expanduser(os.path.join("~", ".keras"))
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
models_dir = os.path.join(cache_dir, 'models')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

In [ ]:
!cp ../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5 ~/.keras/models/
!cp ../input/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/
# !cp ../input/keras-pretrained-models/resnet50* ~/.keras/models/

In [ ]:
!ls ~/.keras/models

In [ ]:
df_train = pd.read_csv("../input/dog-breed-identification/labels.csv")
df_test = pd.read_csv("../input/dog-breed-identification/sample_submission.csv")

In [ ]:
df_train.head(10)

In [ ]:
targets_series = pd.Series(df_train["breed"])
one_hot = pd.get_dummies(targets_series, sparse=True)

In [ ]:
one_hot_labels = np.asarray(one_hot)

In [ ]:
one_hot_labels.shape

In [ ]:
im_size = 145

In [ ]:
x_train = []
y_train = []
# x_test = []

In [ ]:
i = 0
for f, breed in tqdm(df_train.values):
    img = cv2.imread("../input/dog-breed-identification/train/{}.jpg".format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1

In [ ]:
# for f in tqdm(df_test["id"].values):
#     img = cv2.imread("../input/dog-breed-identification/test/{}.jpg".format(f))
#     x_test.append(cv2.resize(img, (im_size, im_size)))

In [ ]:
size_generator = 250
x_train_raw = np.array([])
# max_limit = min(size_generator, len(x_train))
# x_interime = np.array(x_train[0:max_limit], np.float32)/255
# np.concatenate([x_train_raw, x_interime])
# del x_train[0:max_limit]
while True:
    max_limit = min(size_generator, len(x_train))
    x_interime = np.array(x_train[0:max_limit], np.float32)/255
    if x_train_raw.shape == (0,):
        x_train_raw = x_interime
    else:
        x_train_raw = np.concatenate([x_train_raw, x_interime])
    del x_train[0:max_limit]
    print(x_train_raw.shape)
    print(len(x_train))
    if len(x_train) == 0:
        break


In [ ]:

# x_train_raw = np.array(x_train, np.float32)/255.
x_train = None
y_train_raw = np.array(y_train, np.uint8)
# x_test = np.array(x_test, np.float32)/255.

In [ ]:
print(x_train_raw.shape)
print(y_train_raw.shape)
# print(x_test.shape)

In [ ]:
num_class = y_train_raw.shape[1]

In [ ]:
# X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)
train_propotion = 0.7
rnd = np.random.random(x_train_raw.shape[0])
train_idx = rnd < train_propotion
valid_idx = rnd >= train_propotion


In [ ]:
x_train_raw.size

In [ ]:
X_train, Y_train, X_valid, Y_valid = (x_train_raw[train_idx], y_train_raw[train_idx], 
                                      x_train_raw[valid_idx], y_train_raw[valid_idx])
x_train_raw = None
y_train_raw = None

In [ ]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(im_size, im_size, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(2048, activation="relu")(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(num_class, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
callbacks_list = [keras.callbacks.EarlyStopping(monitor="val_acc", patience=3, 
                                               verbose=1)]
model.summary()

model.fit(X_train, Y_train, epochs=25, 
          validation_data=(X_valid, Y_valid), verbose=1)